In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Настройки отображения
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:.2f}'.format

# Инициализация путей
root = Path(os.getcwd()).parent.parent
data_raw = root / 'data' / 'raw_data'
data_out = root / 'data' / 'processed_data'

print("Пути инициализированы:")
print(f"Raw data: {data_raw}")
print(f"Processed data: {data_out}")

Пути инициализированы:
Raw data: /data/raw_data
Processed data: /data/processed_data


In [4]:
# Загрузка данных
print("\nЗагружаем сырые данные...")
sessions_raw = pd.read_pickle(data_raw / 'ga_sessions.pkl')
hits_raw = pd.read_pickle(data_raw / 'ga_hits.pkl')
print("\nДанные успешно загружены!")


Загружаем сырые данные...


FileNotFoundError: [Errno 2] No such file or directory: '/data/raw_data/ga_sessions.pkl'

In [5]:
def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")

    # Типы данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())

    # Пропуски
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing / df.shape[0] * 100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")

    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")

    # Примеры
    print(f"\nПервые {sample_size} записей:")
    display(df.head(sample_size))


def analyze_column(df: pd.DataFrame, col: str, max_display: int = 50) -> None:
    """Полный анализ колонки с выводом всех уникальных значений"""
    print(f"\n{'-'*60}")
    print(f"Полный анализ колонки: {col}")

    # Проверка существования колонки
    if col not in df.columns:
        print(f"Колонка {col} не найдена!")
        return

    # Вывод типа данных
    print(f"Тип данных: {df[col].dtype}")

    # Пропуски
    na_count = df[col].isna().sum()
    print(f"Пропуски: {na_count} ({na_count/len(df)*100:.1f}%)")

    # Количество уникальных значений
    unique_count = df[col].nunique(dropna=False)
    print(f"Уникальных значений: {unique_count}")

    # Вывод всех значений для категориальных данных
    if unique_count <= max_display:
        print("\nВсе значения:")
        print(df[col].unique())
    else:
        print(f"\nСлишком много значений (> {max_display}). Примеры:")
        print(df[col].dropna().sample(10).unique())

    # Частотный анализ для числовых колонок
    if pd.api.types.is_numeric_dtype(df[col]):
        print("\nОписательная статистика:")
        print(df[col].describe())
    else:
        print("\nТоп-5 значений:")
        print(df[col].value_counts(dropna=False).head(10))


generate_summary(sessions_raw, "Сырые данные сессий")
for col in sessions_raw.columns:
    analyze_column(sessions_raw, col, max_display=100)

NameError: name 'sessions_raw' is not defined

In [ ]:
# Копируем данные
sessions = sessions_raw.copy()

# Объединяем 'visit_date' и 'visit_time' в 'visit_datetime'
sessions['visit_datetime'] = pd.to_datetime(
    sessions['visit_date'].astype(str) + ' ' + sessions['visit_time'].astype(str),
    errors='coerce'
)

# Удаляем старые столбцы 'visit_date' и 'visit_time'
sessions.drop(columns=['visit_date', 'visit_time'], inplace=True)

# Вставляем 'visit_datetime' после 'client_id'
sessions.insert(sessions.columns.get_loc('client_id') + 1, 'visit_datetime', sessions.pop('visit_datetime'))

# Извлекаем дополнительные данные из 'visit_datetime'
sessions['visit_hour'] = sessions['visit_datetime'].dt.hour
sessions['visit_weekday'] = sessions['visit_datetime'].dt.weekday
sessions['is_weekend'] = sessions['visit_weekday'] >= 5

# Добавляем новые столбцы после 'visit_datetime'
for col in ['is_weekend', 'visit_weekday', 'visit_hour']:
    sessions.insert(sessions.columns.get_loc('visit_datetime') + 1, col, sessions.pop(col))

# Удаляем столбец 'device_model' (99% пропусков)
sessions.drop(columns=['device_model'], inplace=True)

# Выделяем два отдельных признака высоту и ширину экрана
resolution = (
        sessions['device_screen_resolution']
        .str.extract(r'(?P<device_screen_width>\d+)x(?P<device_screen_height>\d+)')
    )

sessions['device_screen_width'] = resolution['device_screen_width'].astype('Int32')
sessions['device_screen_height'] = resolution['device_screen_height'].astype('Int32')

# Удаляем старый столбец 'device_screen_resolution'
sessions.drop(columns=['device_screen_resolution'], inplace=True)

# Добавляем новые столбцы после 'device_brand'
for col in ['device_screen_width', 'device_screen_height']:
    sessions.insert(sessions.columns.get_loc('device_brand') + 1, col, sessions.pop(col))
#================================================================
# Приводим столбцы к правильным типам и заменяем значения
#sessions = sessions.convert_dtypes()
#================================================================
# Заменяем значения в нескольких столбцах на pd.NA

bad = ['(none)', 'nan', '(not set)', 'NaN', 'None', '']

replace_dict = {
    'utm_source':bad,
    'utm_medium': bad,
    'utm_campaign':bad,
    'utm_adcontent': bad,
    'utm_keyword': bad,
    'device_category':bad,
    'device_os': bad,
    'device_browser': bad,
    'device_brand': bad,
    'geo_country': bad,
    'geo_city': bad,
    'device_screen_width':[0],
    'device_screen_height': [0]

}

for col, values in replace_dict.items():
    for value in values:
        sessions[col] = sessions[col].replace(value, pd.NA)
#==========================================================================
# Удаляем строки с пропущенным значением (количество <100 строк)
sessions.dropna(subset=['utm_source'], inplace=True)
sessions.dropna(subset=['device_browser'], inplace=True)
sessions.dropna(subset=['device_screen_width'], inplace=True)
sessions.dropna(subset=['device_screen_height'], inplace=True)
#==========================================================================
# Заполняем пропуски информации о девайсах исходя из его категории

def fill_missing(df, categories, fields, mask_category):

    for category in categories:
        for field in fields:
            placeholder = f'noname_{category}_{field.split("_")[1]}'
            mask = (df[mask_category] == category) & (df[field].isna())
            df.loc[mask, field] = placeholder

    return df

device_categories = ['mobile', 'desktop', 'tablet']
device_fields = ['device_os', 'device_brand', 'device_browser']
device_mask_category = 'device_category'

sessions = fill_missing(sessions, device_categories, device_fields, device_mask_category)
'''EDA
#==========================================================================
# Профильтруем девайсы
#==========================================================================
# Анализ
plt.figure(figsize=(12, 6))
sns.countplot(data=sessions, y='device_os', order=sessions['device_os'].value_counts().index[:20])
plt.title('Топ-20')
plt.xlabel('Частота')
plt.ylabel('Значение')
plt.grid(True)
plt.show()

threshold_device_os = 100  # минимальное количество упоминаний

# Считаем, сколько раз встречается (нужно поправить переменную)
city_counts = sessions['device_os'].value_counts()

# Определяем редкие
rare_cities = city_counts[city_counts < threshold_device_os].index

# Создаём маску только для редких
rare_mask = sessions['device_os'].isin(rare_cities)

# Логика замены редких
sessions.loc[rare_mask & (sessions['device_category'] == 'mobile'), 'device_os'] = 'other_mobile_os'
sessions.loc[rare_mask & (sessions['device_category'] == 'desktop'), 'device_os'] = 'other_desktop_os'
sessions.loc[rare_mask & (sessions['device_category'] == 'tablet'), 'device_os'] = 'other_tablet_os'

# Анализ
plt.figure(figsize=(12, 6))
sns.countplot(data=sessions, y='device_os', order=sessions['device_os'].value_counts().index[:20])
plt.title('Топ-20')
plt.xlabel('Частота')
plt.ylabel('Значение')
plt.grid(True)
plt.show()

# Оставляем только те значения, которые встречаются часто
common_values = city_counts[city_counts >= 4969].index

# Фильтруем строки
sessions = sessions[sessions['device_os'].isin(common_values)].copy()

# Анализ
plt.figure(figsize=(12, 6))
sns.countplot(data=sessions, y='device_os', order=sessions['device_os'].value_counts().index[:20])
plt.title('Топ-20')
plt.xlabel('Частота')
plt.ylabel('Значение')
plt.grid(True)
plt.show()
#==========================================================================
# Анализ
plt.figure(figsize=(12, 6))
sns.countplot(data=sessions, y='device_brand', order=sessions['device_brand'].value_counts().index[:20])
plt.title('Топ-20')
plt.xlabel('Частота')
plt.ylabel('Значение')
plt.grid(True)
plt.show()

threshold_device_brand = 19500  # минимальное количество упоминаний

# Считаем, сколько раз встречается
city_counts = sessions['device_brand'].value_counts()

# Определяем редкие
rare_cities = city_counts[city_counts < threshold_device_brand].index

# Создаём маску только для редких
rare_mask = sessions['device_brand'].isin(rare_cities)

# Логика замены редких
sessions.loc[rare_mask & (sessions['device_category'] == 'mobile'), 'device_brand'] = 'other_mobile_brand'
sessions.loc[rare_mask & (sessions['device_category'] == 'desktop'), 'device_brand'] = 'other_desktop_brand'
sessions.loc[rare_mask & (sessions['device_category'] == 'tablet'), 'device_brand'] = 'other_tablet_brand'

# Анализ
plt.figure(figsize=(12, 6))
sns.countplot(data=sessions, y='device_brand', order=sessions['device_brand'].value_counts().index[:20])
plt.title('Топ-20')
plt.xlabel('Частота')
plt.ylabel('Значение')
plt.grid(True)
plt.show()

# Оставляем только те значения, которые встречаются часто
common_values = city_counts[city_counts >= 129000].index

# Фильтруем строки
sessions = sessions[sessions['device_brand'].isin(common_values)].copy()

# Анализ
plt.figure(figsize=(12, 6))
sns.countplot(data=sessions, y='device_brand', order=sessions['device_brand'].value_counts().index[:20])
plt.title('Топ-20')
plt.xlabel('Частота')
plt.ylabel('Значение')
plt.grid(True)
plt.show()
#==========================================================================
# Анализ
plt.figure(figsize=(12, 6))
sns.countplot(data=sessions, y='device_browser', order=sessions['device_browser'].value_counts().index[:20])
plt.title('Топ-20')
plt.xlabel('Частота')
plt.ylabel('Значение')
plt.grid(True)
plt.show()

threshold_device_browser = 1000  # минимальное количество упоминаний

# Считаем, сколько раз встречается
city_counts = sessions['device_browser'].value_counts()

# Определяем редкие
rare_cities = city_counts[city_counts < threshold_device_browser].index

# Создаём маску только для редких
rare_mask = sessions['device_browser'].isin(rare_cities)

# Логика замены редких
sessions.loc[rare_mask & (sessions['device_category'] == 'mobile'), 'device_browser'] = 'other_mobile_browser'
sessions.loc[rare_mask & (sessions['device_category'] == 'desktop'), 'device_browser'] = 'other_desktop_browser'
sessions.loc[rare_mask & (sessions['device_category'] == 'tablet'), 'device_browser'] = 'other_tablet_browser'

# Анализ
plt.figure(figsize=(12, 6))
sns.countplot(data=sessions, y='device_browser', order=sessions['device_browser'].value_counts().index[:20])
plt.title('Топ-20')
plt.xlabel('Частота')
plt.ylabel('Значение')
plt.grid(True)
plt.show()

# Оставляем только те значения, которые встречаются часто
common_values = city_counts[city_counts >= 14716].index

# Фильтруем строки
sessions = sessions[sessions['device_browser'].isin(common_values)].copy()

# Анализ
plt.figure(figsize=(12, 6))
sns.countplot(data=sessions, y='device_browser', order=sessions['device_browser'].value_counts().index[:20])
plt.title('Топ-20')
plt.xlabel('Частота')
plt.ylabel('Значение')
plt.grid(True)
plt.show()
'''
#===========================================================================
# Заполняем пропуски геопозиции
#===========================================================================
def mark_abroad(df: pd.DataFrame, country_col: str = 'geo_country') -> pd.DataFrame:
    """
    Если значение в столбце country_col не Россия — подставляется 'abroad'.
    Пропуски остаются как есть.
    """
    df[country_col] = df[country_col].apply(
        lambda x: 'russia' if isinstance(x, str) and x.strip().lower() == 'russia'
        else ('abroad' if pd.notna(x) else x)
    )
    return df

sessions = mark_abroad(sessions, country_col='geo_country')

sessions['geo_country'] = sessions['geo_country'].fillna('missing')

geo_categories = ['russia', 'abroad', 'missing']
geo_fields = ['geo_city']
geo_mask_category = 'geo_country'

sessions = fill_missing(sessions, geo_categories, geo_fields, geo_mask_category)
'''EDA
#===========================================================================
# Отфильтруем города

threshold_city = 20000  # минимальное количество упоминаний

# Считаем, сколько раз встречается каждый город
city_counts = sessions['geo_city'].value_counts()

# Определяем редкие города
rare_cities = city_counts[city_counts < threshold_city].index

# Создаём маску только для редких городов
rare_mask = sessions['geo_city'].isin(rare_cities)

# Логика замены редких городов по стране
sessions.loc[rare_mask & (sessions['geo_country'] == 'russia'), 'geo_city'] = 'other_russian_city'
sessions.loc[rare_mask & (sessions['geo_country'] == 'abroad'), 'geo_city'] = 'other_abroad_city'
sessions.loc[rare_mask & (sessions['geo_country'] == 'missing'), 'geo_city'] = 'other_missing_city'
'''
#===========================================================================
# Заполняем пропуски в utm_*

sessions['utm_medium'] = sessions['utm_medium'].fillna('missing')
sessions['utm_campaign'] = sessions['utm_campaign'].fillna('missing')
sessions['utm_adcontent'] = sessions['utm_adcontent'].fillna('missing')
sessions['utm_keyword'] = sessions['utm_keyword'].fillna('missing')
'''EDA
#==========================================================================
# Фильтруем visit_number
#==========================================================================
# Проанализируем visit_number
plt.figure()
sns.histplot(sessions['visit_number'], bins=50, kde=True)
plt.xlabel('visit_number')
plt.ylabel('Частота')
plt.title('Распределение visit_number')
plt.show()

# Фильтрация численных выбросов в hit_number
sessions = sessions[sessions['visit_number'] <= 8].copy()

# Проанализируем visit_number
plt.figure()
sns.histplot(sessions['visit_number'], bins=50, kde=True)
plt.xlabel('visit_number')
plt.ylabel('Частота')
plt.title('Распределение visit_number')
plt.show()
#=========================================================================
# Отфильтруем разрешения
#=========================================================================
# Вычисляем соотношение сторон
sessions['aspect_ratio'] = sessions['device_screen_width'] / sessions['device_screen_height']

# Фильтруем строки с допустимым соотношением
sessions = sessions[
    (sessions['aspect_ratio'] >= 0.3) &
    (sessions['aspect_ratio'] <= 2.5)
].copy()

# Удаляем вспомогательную колонку
#sessions.drop(columns='aspect_ratio', inplace=True)
'''
'''EDA
# Фильтрация

def fill_and_group_rare_bulk(
    df: pd.DataFrame,
    cols: list[str],
    min_freq: float = 0.01,
    fill_label: str = 'missing',
    rare_label: str = '0ther'
) -> pd.DataFrame:
    """
    Для каждого столбца из cols:
      1. Заменяет NaN на fill_label.
      2. Считает частоту каждой категории.
      3. Все значения с долей < min_freq заменяет на rare_label.
    """
    for col in cols:
        # 1. Импутация пропусков
        df[col] = df[col].fillna(fill_label)

        # 2. Вычисляем относительные частоты
        freqs = df[col].value_counts(normalize=True)

        # 3. Определяем редкие лейблы
        rare = freqs[freqs < min_freq].index

        # 4. Заменяем все редкие на rare_label
        df[col] = df[col].replace(rare, rare_label)

    return df

# Использование:
categorical_cols = ['device_brand', 'device_browser']
sessions = fill_and_group_rare_bulk(
    sessions,
    cols=categorical_cols,
    min_freq=0.01,
    fill_label='missing',
    rare_label='other'
)
'''

# Приводим столбцы к правильным типам и заменяем значения
sessions = sessions.convert_dtypes()

# Генерация отчета
generate_summary(sessions, "Данные сессий")
for col in sessions.columns:
    analyze_column(sessions, col, max_display=100)



================================================== ДАННЫЕ СЕССИЙ ==================================================
Общее количество записей: 1,859,909
Количество признаков: 20

Типы данных:
                count
string[python]     13
Int32               4
datetime64[ns]      1
boolean             1
Int64               1

Пропущенных значений нет

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_datetime,visit_hour,visit_weekday,is_weekend,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_height,device_screen_width,device_browser,geo_country,geo_city
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust
1,905544597018549464.1636867...,210838531.1636867288,2021-11-14 08:21:30,8,6,True,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,854,385,Samsung Internet,russia,Moscow
2,9055446045651783499.164064...,2108385331.1640648523,2021-12-28 02:42:06,2,1,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Krasnoyarsk



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1859909

Слишком много значений (> 100). Примеры:
<StringArray>
['4946136102644513171.1625061779.1625061779',
 '8754195544848095441.1627021166.1627021166',
 '1215887414710193308.1621839004.1621839004',
 '2307084720240748947.1622043035.1622043035',
 '4641228428612588113.1625674323.1625674323',
 '7726630404598101397.1624077716.1624077716',
  '517779101102387567.1633026298.1633026298',
 '3016393501116773335.1625076695.1625076695',
 '8495021099199687345.1631513266.1631513266',
 '9213139802897772831.1639706913.1639706913']
Length: 10, dtype: string

Топ-5 значений:
session_id
9055434745589932991.1637753792.1637753792    1
9055645813163970503.1633964004.1633964004    1
9055447046360770272.1622255328.1622255328    1
9055447046360770272.1622255345.1622255345    1
9055447192389856083.1622453074.1622453074    1
9055455318486370642.1640843788.1

In [ ]:
generate_summary(hits_raw, "Сырые данные событий")
for col in hits_raw.columns:
    analyze_column(hits_raw, col, max_display=100)


================================================== СЫРЫЕ ДАННЫЕ СОБЫТИЙ ==================================================
Общее количество записей: 15,726,470
Количество признаков: 11

Типы данных:
         count
object       9
float64      1
int64        1

Пропущенные значения:
                count      %
event_value  15726470 100.00
hit_time      9160322  58.25
hit_referer   6274804  39.90
event_label   3760184  23.91

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.164025...,2021-12-23,597864.00,30,event,NaN,sberauto.com/cars?utm_sour...,quiz,quiz_show,NaN,None
1,7750352294969115059.164027...,2021-12-23,597331.00,41,event,NaN,sberauto.com/cars/fiat?cit...,quiz,quiz_show,NaN,None
2,885342191847998240.1640235...,2021-12-23,796252.00,49,event,NaN,sberauto.com/cars/all/volk...,quiz,quiz_show,NaN,None



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: object
Пропуски: 0 (0.0%)
Уникальных значений: 1734610

Слишком много значений (> 100). Примеры:
['6132824421085963681.1622451397.1622451397'
 '3705639798257204521.1621876830.1621876830'
 '9113690612536257485.1627990989.1627990989'
 '5995252199090427391.1638536703.1638536703'
 '2662045840061467574.1622412515.1622412515'
 '4505582765727212900.1632298414.1632298429'
 '1598186795953179375.1635928815.1635928815'
 '3026453104794769421.1621858317.1621858317'
 '207293613179146509.1639977229.1639977229'
 '8799003349520186059.1637841751.1637841751']

Топ-5 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.1629319807.1629319807    496
686125592720823356.1634

In [ ]:
# Копируем данные
hits = hits_raw.copy()

# Удаляем столбецы 'event_value', 'hit_type' (неинформативны)
hits.drop(columns=['event_value', 'hit_type'], inplace=True)
#==============================================================
# Приводим hit_date и hit_time
hits['hit_date_dt'] = pd.to_datetime(hits['hit_date'], format='%Y-%m-%d', errors='coerce')

hits['hit_time_td'] = pd.to_timedelta(hits['hit_time'], unit='ms')

# Удаляем старые столбцы 'hit_date' и 'hit_time'
hits.drop(columns=['hit_date', 'hit_time'], inplace=True)
#=============================================================
# Добавляем новые столбцы после 'device_brand'
for col in ['hit_time_td', 'hit_date_dt']:
    hits.insert(hits.columns.get_loc('session_id') + 1, col, hits.pop(col))
#=============================================================
# Приводим столбцы к правильным типам и заменяем значения
hits = hits.convert_dtypes()
'''EDA
#=============================================================
# Проанализируем hit_number
plt.figure()
sns.histplot(hits['hit_number'], bins=50, kde=True)
plt.xlabel('hit_number')
plt.ylabel('Частота')
plt.title('Распределение hit_number')
plt.show()

# Фильтрация численных выбросов в hit_number
hits = hits[hits['hit_number'] <= 100].copy()

# Проанализируем hit_number
plt.figure()
sns.histplot(hits['hit_number'], bins=50, kde=True)
plt.xlabel('hit_number')
plt.ylabel('Частота')
plt.title('Распределение hit_number')
plt.show()
'''
#================================================================
# Заполняем пропуски, когда hit_number = 1
mask = (hits['hit_number'] == 1) & (hits['hit_time_td'].isna())
hits.loc[mask, 'hit_time_td'] = pd.Timedelta(0)

# Заполняем пропуски hit_time_dt с учетом hit_number
# Вычисляем моду (самое частое) для каждой группы hit_number
mode_per_hit = (
    hits
    .groupby('hit_number')['hit_time_td']
    .agg(lambda x: x.mode(dropna=True).iloc[0] if not x.mode(dropna=True).empty else pd.Timedelta(0))
)

# Заполняем пропуски в исходном столбце соответствующими модами
hits['hit_time_td'] = hits['hit_time_td'].fillna(
    hits['hit_number'].map(mode_per_hit)
)
#=================================================================
# Заполняем пропуски в категориальных столбцах
hits['hit_referer'] = hits['hit_referer'].fillna('missing')
hits['event_label'] = hits['event_label'].fillna('missing')
#=================================================================
# Объявляем целевые действия
target_actions = [
    # Отправка форм и заявок
    'sub_submit_success', 'sub_submit_error',          # sub_submit
    'form_request_call_sent',                          # phone
    'sub_callback_submit_click',                       # sub_button_click
    'sub_car_request_submit_click',                    # sub_button_click
    'sub_custom_question_submit_click',                # sub_button_click
    'sub_car_claim_submit_click',                      # sub_button_click

    # Обратный звонок и контакт
    'click_on_request_call', 'click_on_phone',          # phone
    'phone_entered_on_form_request_call',               # phone
    'showed_form_request_call',                         # phone

    # Чат и онлайн-диалог
    'start_chat', 'chat requested',                     # chat, jivosite
    'user_message', 'client initiate chat',             # chat, jivosite
    'callback requested',                               # jivosite
    'offline form shown',                               # jivosite

    # Аутентификация (сильный признак санкционированного взаимодействия)
    'phone_auth_success', 'sber_id_auth_success',       # auth

    # Кредит и рассчет
    'calculate',                                        # credit_landing_pos
    'request', 'request_success',                       # credit_landing_pos

    # Клики по покупке/оформлению подписки
    'click_on_subscription',                            # header, main_banners
    'click_auto_subscription',                          # main_services
    'click_free_car_selection',                         # main_services, chat
    'click_buy_auto',                                   # main_advantages, main_services
    'click_auto',                                       # main_services
    'click_car_selection',                              # main_services
    'click_car_buyback',                                # header, main_services
    'click_setelem_credit',                             # main_services
    'click_pos_credit',                                 # main_services

    # Публикация объявления (для частных продавцов)
    'success_ad_creation',                              # add_ad_publish_success

    # Запросы по VIN, цене, городу — сильный признак готовности
    'forward_to_price', 'sap_search_form_cost_to',      # add_ad_city, sap_search_form
    'sap_search_form_cost_from',                       # sap_search__form

    # Переходы из листинга и карточки
    'go_to_car_card',                                   # listing_ads, greenday_listing_ads
    'sub_view_cars_click',                              # sub_button_click, greenday_gtm.triggergroup

    # Клики из “подписаться”
    'sub_open_dialog_click',                            # sub_button_click, greenday_sub_button_click

    # Дополнительные целевые события
    'request',                                          # credit_landing_pos (повтор)
    'order_call',                                       # если встречается в схеме
]

hits['is_target'] = hits['event_action'].isin(target_actions).astype('Int32')
#============================================================
'''EDA
# Группируем по session_id и считаем основные метрики
agg = hits.groupby('session_id').agg(
    events_count        = ('hit_number',    'max'),
    unique_pages        = ('hit_page_path', 'nunique'),
    target_events       = ('is_target',     'sum'),
    first_hit_time      = ('hit_datetime',  'min'),
    last_hit_time       = ('hit_datetime',  'max'),
)

# Вычисляем длительность и средний интервал
agg['session_duration_sec'] = (
    (agg['last_hit_time'] - agg['first_hit_time'])
     .dt.total_seconds()
     .fillna(0)
)

# Средний интервал между событиями
agg['avg_time_between_hits'] = agg.apply(
    lambda row: row['session_duration_sec'] / (row['events_count'] - 1)
                if row['events_count'] > 1 else 0,
    axis=1
)

# Час начала и конца сессии
agg['first_hit_hour'] = agg['first_hit_time'].dt.hour
agg['last_hit_hour']  = agg['last_hit_time'] .dt.hour

# Оставляем только то, что нужно
agg = agg.drop(columns=['first_hit_time', 'last_hit_time']).reset_index()
'''

generate_summary(hits, "Данные событий")
for col in hits.columns:
    analyze_column(hits, col, max_display=100)



================================================== ДАННЫЕ СОБЫТИЙ ==================================================
Общее количество записей: 15,726,470
Количество признаков: 10

Типы данных:
                 count
string[python]       6
datetime64[ns]       1
timedelta64[ns]      1
Int64                1
Int32                1

Пропущенных значений нет

Дубликаты: 94 (0.00%)

Первые 3 записей:


,session_id,hit_date_dt,hit_time_td,hit_number,hit_referer,hit_page_path,event_category,event_action,event_label,is_target
0,5639623078712724064.164025...,2021-12-23,0 days 00:09:57.864000,30,missing,sberauto.com/cars?utm_sour...,quiz,quiz_show,missing,0
1,7750352294969115059.164027...,2021-12-23,0 days 00:09:57.331000,41,missing,sberauto.com/cars/fiat?cit...,quiz,quiz_show,missing,0
2,885342191847998240.1640235...,2021-12-23,0 days 00:13:16.252000,49,missing,sberauto.com/cars/all/volk...,quiz,quiz_show,missing,0



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1734610

Слишком много значений (> 100). Примеры:
<StringArray>
['7568840835237355561.1640701993.1640701993',
 '5917833386338776092.1621856603.1621856603',
 '7281935112898102993.1639728850.1639728850',
 '4406787303764286375.1636981675.1636981675',
 '7499288298054362469.1640375653.1640375653',
 '4100063803461557479.1640193128.1640193128',
 '5450810213227363022.1635593155.1635593155',
 '3089800483700658948.1640944388.1640944388',
 '2687079082719092883.1624940691.1624940691',
 '8317718737477043929.1626212064.1626212064']
Length: 10, dtype: string

Топ-5 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.16

In [ ]:
# Копируем обработанные сессии и события
hits_p = hits.copy()
sessions_p = sessions.copy()

# Соеденяем в один большой датасет
full_raw = sessions_p.merge(
    hits_p,
    how='inner',
    on='session_id',
    suffixes=('_sess','_hit')
)

generate_summary(full_raw, "Общие данные")
for col in full_raw.columns:
    analyze_column(full_raw, col, max_display=100)


================================================== ОБЩИЕ ДАННЫЕ ==================================================
Общее количество записей: 15,684,368
Количество признаков: 29

Типы данных:
                 count
string[python]      18
Int32                5
datetime64[ns]       2
Int64                2
boolean              1
timedelta64[ns]      1

Пропущенных значений нет

Дубликаты: 94 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_datetime,visit_hour,visit_weekday,is_weekend,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_height,device_screen_width,device_browser,geo_country,geo_city,hit_date_dt,hit_time_td,hit_number,hit_referer,hit_page_path,event_category,event_action,event_label,is_target
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust,2021-11-24,0 days 00:00:03.665000,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0
1,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,False,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust,2021-11-24,0 days 00:00:46.592000,4,missing,podpiska.sberauto.com/,sub_button_click,sub_view_cars_click,vodKSlUobUWTVlgsJqdI,1
2,905544597018549464.1636867...,210838531.1636867288,2021-11-14 08:21:30,8,6,True,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,854,385,Samsung Internet,russia,Moscow,2021-11-14,0 days 00:00:00.921000,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1732170

Слишком много значений (> 100). Примеры:
<StringArray>
['5040141065470106555.1630837305.1630837305',
 '2541671324246252967.1635687847.1635687847',
 '7919861142173653074.1638799444.1638799444',
 '7187266805265185866.1640610892.1640610892',
 '8561058068213545003.1633433643.1633433672',
 '8935584576548602635.1638105873.1638105873',
 '5788734202793670412.1622023323.1622023323',
 '5483187708214567579.1632342036.1632342036',
  '605299354785383630.1623295186.1623295186',
 '7375337728017053912.1627273355.1627273355']
Length: 10, dtype: string

Топ-5 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.16

In [ ]:
# Копируем датасет
full = full_raw.copy()
#========================================================
#Удалим дубликаты
full.drop_duplicates(inplace=True, keep='first')
#========================================================
# Приводим типы
full['visit_number'] = full['visit_number'].astype('Int32')
full['hit_number'] = full['hit_number'].astype('Int32')
full['is_weekend'] = full['is_weekend'].astype('Int32')
#full['aspect_ratio'] = full['aspect_ratio'].astype('Float32')
'''EDA
#========================================================
# Проанализируем 'hit_date_dt' и 'visit_datetime'
# Извлечём из visit_datetime только дату
full['visit_date_only'] = full['visit_datetime'].dt.normalize()

# Сравним с hit_date_dt
mismatch = full.loc[
    full['visit_date_only'] != full['hit_date_dt'].dt.normalize(),
    ['visit_datetime', 'hit_date_dt']
]

# Посмотрим, есть ли расхождения
if mismatch.empty:
    print("Все даты совпадают.")
else:
    print(f"Найдены несовпадения: {len(mismatch)} записей")
    display(mismatch.head(10))

total = len(full)
bad = len(mismatch)
print(f"Доля несовпадений: {bad/total:.4%}")

#Удалим вспомогательную колонку 'visit_date_only'
full.drop(columns=['visit_date_only'], inplace=True)

# Удалим колонку 'hit_date_dt', потому что даты визита и даты событий совпадают
full.drop(columns=['hit_date_dt'], inplace=True)
'''
#=============================================================
generate_summary(full, "Общие данные")
for col in full.columns:
    analyze_column(full, col, max_display=100)


================================================== ОБЩИЕ ДАННЫЕ ==================================================
Общее количество записей: 15,684,274
Количество признаков: 29

Типы данных:
                 count
string[python]      18
Int32                8
datetime64[ns]       2
timedelta64[ns]      1

Пропущенных значений нет

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_datetime,visit_hour,visit_weekday,is_weekend,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_height,device_screen_width,device_browser,geo_country,geo_city,hit_date_dt,hit_time_td,hit_number,hit_referer,hit_page_path,event_category,event_action,event_label,is_target
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,0,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust,2021-11-24,0 days 00:00:03.665000,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0
1,9055434745589932991.163775...,2108382700.1637753791,2021-11-24 14:36:32,14,2,0,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,Zlatoust,2021-11-24,0 days 00:00:46.592000,4,missing,podpiska.sberauto.com/,sub_button_click,sub_view_cars_click,vodKSlUobUWTVlgsJqdI,1
2,905544597018549464.1636867...,210838531.1636867288,2021-11-14 08:21:30,8,6,1,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,854,385,Samsung Internet,russia,Moscow,2021-11-14,0 days 00:00:00.921000,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0



------------------------------------------------------------
Полный анализ колонки: session_id
Тип данных: string
Пропуски: 0 (0.0%)
Уникальных значений: 1732170

Слишком много значений (> 100). Примеры:
<StringArray>
['6243807668807907440.1629993074.1629993074',
 '4844457841456745453.1633848305.1633848305',
 '1245706805721119559.1632320116.1632320116',
  '332826049719178591.1635915103.1635915103',
 '1217008495667241246.1638380646.1638380646',
 '1879080530430621566.1621849982.1621849982',
 '4533590921764092453.1622175458.1622175458',
 '9105085443705982638.1638715054.1638715054',
 '8719057463911714578.1621954323.1621954494',
 '8234297789546197024.1622110239.1622110239']
Length: 10, dtype: string

Топ-5 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
686125592720823356.163

In [ ]:
print("\nСохраняем обработанные данные...")
data_out.mkdir(parents=True, exist_ok=True)
full.to_pickle(data_out / 'data_processed.pkl')
print(f"Данные сохранены в: {data_out}")


Сохраняем обработанные данные...
Данные сохранены в: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/processed_data


In [ ]:
'''EDA
# 1) Агрегация по session_id
session_conv = (
    test
    .groupby('session_id', as_index=False)
    .agg(
        converted=('is_target', 'max'),
        hits_total=('hit_number', 'count')
    )
)

# 2) Расчёт конверсии на уровне сессий
total_sessions = session_conv.shape[0]
conv_sessions  = session_conv['converted'].sum()
cr_sessions    = conv_sessions / total_sessions

print(f"Сессий всего: {total_sessions}")
print(f"Конвертированных сессий: {conv_sessions}")
print(f"CR (сессии): {cr_sessions:.2%}")

counts_sess = session_conv['converted'].value_counts().sort_index()
plt.figure()
plt.bar(counts_sess.index.astype(str), counts_sess.values)
plt.xlabel('converted (session)')
plt.ylabel('Количество сессий')
plt.title('Распределение конвертированных и не конвертированных сессий')
plt.show()

# Предполагаем, что в test есть все сессионные признаки для одного хита на сессию
sessions_meta = test.drop_duplicates('session_id')[
    ['session_id', 'utm_source', 'device_category', 'geo_city']
]
session_conv = session_conv.merge(sessions_meta, on='session_id', how='left')

# Пример: CR по источнику
cr_by_source = (
    session_conv
    .groupby('utm_source', as_index=False)
    .agg(sessions=('session_id','count'),
         conv=('converted','sum'))
)
cr_by_source['CR'] = cr_by_source['conv'] / cr_by_source['sessions']
print(cr_by_source.sort_values('CR', ascending=False).head(10))
'''